# Version finale

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

In [2]:


# Load and preprocess the dataset
df = pd.read_csv('./all_buildings_data.csv')

# Drop unnecessary columns
df = df.drop(columns=['Unnamed: 0', 'building'])

# Extract features (aggregate power) and targets (appliance power consumption)
X = df['aggregate'].values.reshape(-1, 1)
y = df.drop(columns=['aggregate']).values  # All appliance columns as targets

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)
joblib.dump(scaler, 'scaler.pkl')


In [4]:

# Create windows of data
def create_windows(data, targets, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(targets[i + window_size])
    return np.array(X), np.array(y)

window_size = 60  # 60 points of time in our window

# Prepare the data
X, y = create_windows(X, y, window_size)

# Reshape X for Conv1D input
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:

# Define the multi-output Seq2Point model
def multi_output_seq2point_model(input_shape, num_outputs):
    model = Sequential()
    model.add(Conv1D(30, 10, activation='relu', input_shape=input_shape, padding='same'))
    model.add(Conv1D(30, 8, activation='relu', padding='same'))
    model.add(Conv1D(40, 6, activation='relu', padding='same'))
    model.add(Conv1D(50, 5, activation='relu', padding='same'))
    model.add(Conv1D(50, 5, activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(num_outputs))  # Output layer for multi-output regression
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

    # Create the model
num_appliances = y_train.shape[1]
model = multi_output_seq2point_model((X_train.shape[1], 1), num_appliances)

In [6]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50
2008/2008 [==============================] - 157s 76ms/step - loss: 4896.4839 - mae: 24.6621 - val_loss: 3758.7856 - val_mae: 20.3081
Epoch 2/50
2008/2008 [==============================] - 152s 76ms/step - loss: 3862.5725 - mae: 20.9599 - val_loss: 4026.1636 - val_mae: 20.5106
Epoch 3/50
2008/2008 [==============================] - 152s 76ms/step - loss: 3528.7781 - mae: 19.6298 - val_loss: 3268.3420 - val_mae: 20.3092
Epoch 4/50
2008/2008 [==============================] - 155s 77ms/step - loss: 3294.0637 - mae: 19.0670 - val_loss: 3104.5808 - val_mae: 19.0075
Epoch 5/50
2008/2008 [==============================] - 154s 77ms/step - loss: 3012.6169 - mae: 18.1298 - val_loss: 3090.0759 - val_mae: 17.7441
Epoch 6/50
2008/2008 [==============================] - 154s 77ms/step - loss: 2865.5784 - mae: 17.6248 - val_loss: 3004.3115 - val_mae: 18.0608
Epoch 7/50
2008/2008 [==============================] - 155s 77ms/step - loss: 2688.3262 - mae: 16.9520 - val_loss: 2783.4927 - va

In [7]:
loss, mae = model.evaluate(X_test, y_test, verbose=1)
print(f'Model Mean Absolute Error: {mae}')


558/558 [==============================] - 8s 13ms/step - loss: 2582.9041 - mae: 13.9785
Model Mean Absolute Error: 13.978487968444824


In [13]:
# List of appliance names
appliances = [
    'laptop computer', 'television', 'light', 'HTPC', 'food processor', 'toasted sandwich maker',
    'toaster', 'microwave', 'computer monitor', 'audio system', 'audio amplifier', 'broadband router',
    'ethernet switch', 'USB hub', 'tablet computer charger', 'radio', 'wireless phone charger', 'mobile phone charger',
    'coffee maker', 'computer', 'external hard disk', 'desktop computer', 'printer', 'immersion heater',
    'security alarm', 'projector', 'server computer', 'running machine', 'network attached storage', 'fridge',
    'air conditioner'
]


# Function to predict appliance consumption from aggregate
def predict_appliance_consumption(aggregate_window, scaler, window_size):
    # Standardize the input features
    aggregate_window = np.array(aggregate_window).reshape(-1, 1)
    aggregate_window = scaler.transform(aggregate_window)
    aggregate_window = aggregate_window.reshape((1, window_size, 1))

    # Predict using the trained model
    predictions = model.predict(aggregate_window)
    # Clip negative values to zero
    predictions = np.clip(predictions, 0, None)
    return predictions

# Example usage (a new window of aggregate power consumption data)
aggregate_window = [
    100, 52.31, 1.0, 135.28, 2.0, 1.0, 1.0, 0.0, 1.0, 4.89, 51.10, 0.0, 6.0,
    51.44, 51.44, 50.52, 135.28, 1.0, 0.0, 0.0, 13.88, 13.88, 57.26, 50.93,
    102.20, 102.20, 135.28, 135.28, 135.28, 135.28, 0.0, 135.28, 99.8, 98.7,
    102.3, 101.2, 100.5, 102.0, 105.0, 99.9, 98.4, 102.1, 101.3, 100.4, 100.7,
    102.8, 101.5, 100.3, 101.7, 100.9, 102.2, 100.6, 99.5, 101.4, 100.2, 100.1,
    99.6, 98.5, 100.0, 99.7
]

predictions = predict_appliance_consumption(aggregate_window, scaler, window_size)


# Map appliance names to their predicted consumption values
predicted_consumption = dict(zip(appliances, predictions[0]))

print("Predicted appliance consumption:")
for appliance, consumption in predicted_consumption.items():
    print(f"{appliance}: {consumption}")


1/1 [==============================] - 0s 25ms/step
Predicted appliance consumption:
laptop computer: 4.010241508483887
television: 3.8627007007598877
light: 106.19120788574219
HTPC: 3.4630422592163086
food processor: 0.0
toasted sandwich maker: 0.0
toaster: 0.0
microwave: 0.17969262599945068
computer monitor: 2.938211679458618
audio system: 3.760430335998535
audio amplifier: 6.218283653259277
broadband router: 6.184991836547852
ethernet switch: 3.972405433654785
USB hub: 4.020544052124023
tablet computer charger: 0.4330313205718994
radio: 106.18269348144531
wireless phone charger: 1.45587956905365
mobile phone charger: 0.5068104267120361
coffee maker: 0.7939611077308655
computer: 13.164216041564941
external hard disk: 13.138588905334473
desktop computer: 5.918463706970215
printer: 3.4136481285095215
immersion heater: 125.8182144165039
security alarm: 126.0098876953125
projector: 106.19452667236328
server computer: 107.14362335205078
running machine: 107.1309585571289
network attached 

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")


558/558 [==============================] - 7s 13ms/step
Mean Absolute Error (MAE): 13.978487983944513
Mean Squared Error (MSE): 2582.9044247016914
R-squared (R²): 0.5731571220226468
